# Задание

# Особенности Apache Spark Dataframes
Изучите теоретическую часть.

In [1]:
# Настраиваем переменные окружения для работы с Apache Spark
import os
os.environ["JAVA_HOME"] = "C:/Program Files/Java/jdk-19"
os.environ["SPARK_HOME"] = "C:/Program Files/spark-3.3.1-bin-hadoop3"
# Находим установку Spark
import findspark
findspark.init()
# Подключаем необходимые модули для работы со Spark из Python
from pyspark.sql import SparkSession
# Создаем сессию Spark на локальном компьютере
spark = SparkSession.builder.master("local[*]").getOrCreate()
!wget https://datahub.io/core/population/r/population.json -O ./Data/population.json -q

## Задача 1

Модернизируйте заготовку заменив все участки ```<put your code here>``` на ваш код для того, что бы:
* вычислить прирост населения в каждой стране от года к году (воспользуйтесь функцией [lag](https://spark.apache.org/docs/2.3.0/api/sql/#lag) или [lead](https://spark.apache.org/docs/2.3.0/api/sql/#lead))
* вычислить средний прирост за период с 1990 по 2018 год
* вычислить и вывести на экран страны с трендом на убыль населения с 1990 по 2018 годы


Пример вывода на экран:
```
+------------+-------------------+
|Country Name|              trend|
+------------+-------------------+
|    Bulgaria|-14276.898305084746|
|     Hungary|-3532.2542372881358|
|      Latvia| -3284.830508474576|
+------------+-------------------+
```

In [2]:
from pyspark.sql.functions import lag, col, coalesce, lit, avg, desc
df = spark.read.json("./Data/population.json")

**Выведем схему кадра данных**

In [3]:
df.printSchema()

root
 |-- Country Code: string (nullable = true)
 |-- Country Name: string (nullable = true)
 |-- Value: double (nullable = true)
 |-- Year: long (nullable = true)



**Выведем на экран страны с трендом на убыль населения с 1990 по 2018 годы** 

In [4]:
from pyspark.sql import Window
county_win = Window.partitionBy("Country Name").orderBy("Year")
df = df.where('year > 1990').withColumn("cnt_country", coalesce(col('Value') - lag("Value").over(county_win), lit(0)))
df = df.groupBy("Country Name").agg(avg("cnt_country").alias("trend"))
df.where("trend < 0").orderBy("trend").show(10)

+--------------------+-------------------+
|        Country Name|              trend|
+--------------------+-------------------+
|Central Europe an...|-284198.03571428574|
|             Ukraine|-263498.35714285716|
|  Russian Federation|-148069.64285714287|
|             Romania|-126250.35714285714|
|            Bulgaria|-57404.642857142855|
|             Georgia| -39619.67857142857|
|Bosnia and Herzeg...|-37335.357142857145|
|           Lithuania| -32235.39285714286|
|              Latvia|-25835.964285714286|
|             Belarus|-25375.035714285714|
+--------------------+-------------------+
only showing top 10 rows

